# 矩阵乘法

矩阵乘法是科学计算和深度学习中应用最广泛的运算之一，通常被称为通用矩阵乘法（GEneral Matrix Multiply，简称 GEMM）。在本节中将实现它的计算。

给定 $A\in\mathbb R^{n\times l}$ 和 $B \in\mathbb R^{l\times m}$，如果 $C=AB$ 那么 $C \in\mathbb R^{n\times m}$，且

$$C_{i,j} = \sum_{k=1}^l A_{i,k} B_{k,j}.$$

(fig_matmul_default)=
```{figure} ../img/matmul_default.svg
计算矩阵乘法的原始 $C_{x,y}$
```

下面的方法返回矩阵乘法的计算表达式。

In [1]:
# import d2ltvm
import numpy as np
import tvm
from tvm import te

# Save to the d2ltvm package
def matmul(n, m, l):
    """Return the computing expression of matrix multiplication
    A : n x l matrix
    B : l x m matrix
    C : n x m matrix with C = A B
    """
    k = te.reduce_axis((0, l), name='k')
    A = te.placeholder((n, l), name='A')
    B = te.placeholder((l, m), name='B')
    C = te.compute((n, m),
                    lambda x, y: te.sum(A[x, k] * B[k, y], axis=k),
                    name='C')
    return A, B, C

下面编译方阵乘法模块。

In [2]:
n = 100
A, B, C = matmul(n, n, n)
s = te.create_schedule(C.op)
m = tvm.lower(s, [A, B, C], simple_mode=True)
mod = tvm.build(m)
m["main"]

PrimFunc([A, B, C]) attrs={"from_legacy_te_schedule": (bool)1, "global_symbol": "main", "tir.noalias": (bool)1} {
  for (x, 0, 100) {
    for (y, 0, 100) {
      C[((x*100) + y)] = 0f
      for (k, 0, 100) {
        let cse_var_2 = (x*100)
        let cse_var_1 = (cse_var_2 + y)
        C[cse_var_1] = (C[cse_var_1] + (A[(cse_var_2 + k)]*B[((k*100) + y)]))
      }
    }
  }
}

伪代码只是简单的 3 级嵌套的 for 循环，用于计算矩阵乘法。

然后验证结果。注意，NumPy 可能使用多线程来加速其计算，这可能会由于数值错误而导致略有不同的结果。使用 `assert_allclose` 和相对较大的容错来测试正确性。

In [3]:
from tvm_book.contrib import d2ltvm
a, b, c = d2ltvm.get_abc((100, 100), tvm.nd.array)
mod(a, b, c)
np.testing.assert_allclose(np.dot(a.numpy(), b.numpy()),
                           c.numpy(), atol=1e-5)

## 小结

- 可以用一行代码来表示 TVM 中矩阵乘法的计算。
- 原始矩阵乘法是 3 层嵌套的 for 循环。